## Dataframe Basics

### Let's create two DataFrames one as a Dimension and the other one as a Fact.

In [3]:
personDIM = [
  (123, 'John', 25),
  (234, 'Doe', 27),
  (345, 'Jane', 21),
  (456, 'Jimmy', 45),
]

transFact = [
  (1, 123, 'Soap', 5.00, '2018-01-01'),
  (2, 123 , 'Fruit', 4.67, '2018-01-01'),
  (3, 234 , 'Soap', 5.00, '2018-02-01'),
  (4, 234 , 'Bread', 1.99, '2018-03-01'),
  (5, 234, 'Milk', 4.55, '2018-08-01'),
  (6, 345 , 'Chips', 5.99, '2018-09-01'),
]

In [4]:
personDF = spark.createDataFrame(personDIM, ["id", "name", "age"])

In [5]:
transDF = spark.createDataFrame(transFact, ["id", "person_id", "item", "amount", "purchase_date"])

#### Convert purchase_date to a date format using to_date function

In [7]:
from pyspark.sql.functions import to_date
transDF = transDF.withColumn("purchase_date", to_date("purchase_date" , "yyyy-mm-dd"))

### Perform some DataFrames functions

In [9]:
from pyspark.sql.functions import col
transDF.withColumn("id", col("id") * 2).show()

+---+---------+-----+------+-------------+
 id|person_id| item|amount|purchase_date|
+---+---------+-----+------+-------------+
 2| 123| Soap| 5.0| 2018-01-01|
 4| 123|Fruit| 4.67| 2018-01-01|
 6| 234| Soap| 5.0| 2018-01-01|
 8| 234|Bread| 1.99| 2018-01-01|
 10| 234| Milk| 4.55| 2018-01-01|
 12| 345|Chips| 5.99| 2018-01-01|
+---+---------+-----+------+-------------+

In [10]:
joinExpression = (  transDF["person_id"] == personDF["id"] )
joinType = 'left_outer'
df = personDF.join(transDF,joinExpression,joinType)

In [11]:
display(df)

id,name,age,id,person_id,item,amount,purchase_date
234,Doe,27,3,234,Soap,5.0,2018-01-01
234,Doe,27,4,234,Bread,1.99,2018-01-01
234,Doe,27,5,234,Milk,4.55,2018-01-01
123,John,25,1,123,Soap,5.0,2018-01-01
123,John,25,2,123,Fruit,4.67,2018-01-01
345,Jane,21,6,345,Chips,5.99,2018-01-01
456,Jimmy,45,null,null,null,null,null


In [12]:
from pyspark.sql.functions import sum, desc
df.groupBy("name").agg(sum("amount").alias("Total Spent")).orderBy(desc("Total Spent")).show()

+-----+-----------+
 name|Total Spent|
+-----+-----------+
 Doe| 11.54|
 John| 9.67|
 Jane| 5.99|
Jimmy| null|
+-----+-----------+

### Let's make the following the SQL Way
```
joinExpression = (  transDF["person_id"] == personDF["id"] )
joinType = 'left_outer'
df = personDF.join(transDF,joinExpression,joinType)

from pyspark.sql.functions import sum, desc
df.groupBy("name").agg(sum("amount").alias("Total Spent")).orderBy(desc("Total Spent")).show()

```

In [14]:
transDF.createOrReplaceTempView("transSQL")

In [15]:
personDF.createOrReplaceTempView("personSQL")

In [16]:
dfSQL = spark.sql("""
select *
from personSQL
Left join transSQL on (personSQL.id = transSQL.person_id)
""")

In [17]:
dfSQL.createOrReplaceTempView("dfSQL")

In [18]:
spark.sql("""
select name, sum(amount) as `total amount` from dfSQL
group by name
order by `total amount` desc
""").show()

+-----+------------+
 name|total amount|
+-----+------------+
 Doe| 11.54|
 John| 9.67|
 Jane| 5.99|
Jimmy| null|
+-----+------------+

#### In order to mount s3 to Databricks Execute the following:
```
#%fs mount s3a://Access key ID:Secret access key@bucketname /mnt/mys3
```
#### Make sure that you replace / with %2F in Access Key or Secret Key

In [20]:
%fs ls /mnt/

at com.databricks.backend.daemon.data.client.DbfsClient.send0(DbfsClient.scala:92)
	at com.databricks.backend.daemon.data.client.DbfsClient.sendIdempotent(DbfsClient.scala:54)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystemV1.listStatus(DatabricksFileSystemV1.scala:180)
	at com.databricks.backend.daemon.data.client.DatabricksFileSystem.listStatus(DatabricksFileSystem.scala:143)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$ls$1.apply(DBUtilsCore.scala:83)
	at com.databricks.backend.daemon.dbutils.FSUtils$$anonfun$ls$1.apply(DBUtilsCore.scala:82)
	at com.databricks.backend.daemon.dbutils.FSUtils$.com$databricks$backend$daemon$dbutils$FSUtils$$withFsSafetyCheck(DBUtilsCore.scala:78)
	at com.databricks.backend.daemon.dbutils.FSUtils$.ls(DBUtilsCore.scala:82)
	at com.databricks.dbutils_v1.impl.DbfsUtilsImpl.ls(DbfsUtilsImpl.scala:34)
	at linedd90cd7e18534b84a347c8b8befa258225.$read$$iw$$iw$$iw$$iw$$iw$$iw.<init>(command-346381476745883:1)
	at linedd90cd7e18534b84a347c8b8befa258225.$read$$iw$$iw$$iw$$iw$$iw.<init>(command-346381476745883:44)
	at linedd90cd7e18534b84a347c8b8befa258225.$read$$iw$$iw$$iw$$iw.<init>(command-346381476745883:46)
	at linedd90cd7e18534b84a347c8b8befa258225.$read$$iw$$iw$$iw.<init>(command-346381476745883:48)
	at linedd90cd7e18534b84a347c8b8befa258225.$read$$iw$$iw.<init>(command-346381476745883:50)
	at linedd90cd7e18534b84a347c8b8befa258225.$read$$iw.<init>(command-346381476745883:52)
	at linedd90cd7e18534b84a347c8b8befa258225.$read.<init>(command-346381476745883:54)
	at linedd90cd7e18534b84a347c8b8befa258225.$read$.<init>(command-346381476745883:58)
	at linedd90cd7e18534b84a347c8b8befa258225.$read$.<clinit>(command-346381476745883)
	at linedd90cd7e18534b84a347c8b8befa258225.$eval$.$print$lzycompute(<notebook>:7)
	at linedd90cd7e18534b84a347c8b8befa258225.$eval$.$print(<notebook>:6)
	at linedd90cd7e18534b84a347c8b8befa258225.$eval.$print(<notebook>)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at scala.tools.nsc.interpreter.IMain$ReadEvalPrint.call(IMain.scala:793)
	at scala.tools.nsc.interpreter.IMain$Request.loadAndRun(IMain.scala:1054)
	at scala.tools.nsc.interpreter.IMain$WrappedRequest$$anonfun$loadAndRunReq$1.apply(IMain.scala:645)
	at scala.tools.nsc.interpreter.IMain$WrappedRequest$$anonfun$loadAndRunReq$1.apply(IMain.scala:644)
	at scala.reflect.internal.util.ScalaClassLoader$class.asContext(ScalaClassLoader.scala:31)
	at scala.reflect.internal.util.AbstractFileClassLoader.asContext(AbstractFileClassLoader.scala:19)
	at scala.tools.nsc.interpreter.IMain$WrappedRequest.loadAndRunReq(IMain.scala:644)
	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:576)
	at scala.tools.nsc.interpreter.IMain.interpret(IMain.scala:572)
	at com.databricks.backend.daemon.driver.DriverILoop.execute(DriverILoop.scala:199)
	at com.databricks.backend.daemon.driver.ScalaDriverLocal$$anonfun$repl$1.apply$mcV$sp(ScalaDriverLocal.scala:190)
	at com.databricks.backend.daemon.driver.ScalaDriverLocal$$anonfun$repl$1.apply(ScalaDriverLocal.scala:190)
	at com.databricks.backend.daemon.driver.ScalaDriverLocal$$anonfun$repl$1.apply(ScalaDriverLocal.scala:190)
	at com.databricks.backend.daemon.driver.DriverLocal$TrapExitInternal$.trapExit(DriverLocal.scala:590)
	at com.databricks.backend.daemon.driver.DriverLocal$TrapExit$.apply(DriverLocal.scala:545)
	at com.databricks.backend.daemon.driver.ScalaDriverLocal.repl(ScalaDriverLocal.scala:190)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$8.apply(DriverLocal.scala:323)
	at com.databricks.backend.daemon.driver.DriverLocal$$anonfun$execute$8.apply(DriverLocal.scala:303)
	at com.databricks.logging.UsageLogging$$anonfun$withAttributionContext$1.apply(UsageLogging.scala:235)
	at scala.ut

In [21]:
dfSQL.toPandas()

In [22]:
dfSQL.drop("id").write.format("json").option("path", "/mnt/mys3/demo").save()

In [23]:
df_train = spark.read.format("csv").option("inferSchema", "True").option("header", "True")\
.option("path", "/mnt/mys3/test/train.csv").load()

In [24]:
df_train.show(5)